# MIS 583 Assignment #8: Generative Adversarial Network(GAN)

Before we start, please put your name and ID in following format  
: LASTNAME Firstname, ?000000000   //   e.g.) 陳耀融, M094020099

**Your Answer:**   
Hi I'm XXX, XXXXXXXXXX

This tutorial was originally written by [Nathan Inkawhich](https://github.com/inkawhich) for PyTorch Offical Tutorial.  
This version has been adapted by [Yao-Rong Chen](https://github.com/teacher144123/) for NSYSU MIS583 assigment.


Refrence:
- DCGAN - PyTorch offical tutorial  
https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
- Self-Normalizing Neural Networks  
https://arxiv.org/abs/1706.02515
- Relativistic GAN  
https://arxiv.org/abs/1807.00734

Tutorial or Source code:
- SELU make fnns great again  
https://towardsdatascience.com/selu-make-fnns-great-again-snn-8d61526802a9
- Relativistic GAN offical GitHub **(important)**  
https://github.com/AlexiaJM/RelativisticGAN
- third party Relativistic repo - ririw  
https://github.com/ririw/Relativistic-GAN/blob/master/relgan/trainer.py


## Generative Adversarial Network(GAN)

"GAN and its variations that are now being proposed is the most interesting idea in the last 10 years in ML, in my opinion." LeCun has ever said.

GAN was designed by Ian Goodfellow in 2014 is an approach contains generator and discirminator.

Though originally proposed as a form of generative model for **unsupervised learning**, GANs have also proven useful for **semi-supervised learning**, **fully supervised learning** and **reinforcement learning**.



### What is GAN

GAN is a mixed architecture contains generator and discirminator. They have different goals.  
- Generator generate fake images from latent code, Discriminator classify images into categories.
- The primary goal of Generator is fool the discriminator, make loss of discriminator maximum.
- In contrast, the main goal of discriminator is correctly classify whether a image is real(from original dataset) or fake(made by generator).


In this tutorial, we will cover:
- GAN
- DCGAN
- SELU
- Relativistic GAN
- or any advanced GAN you interested in

This assignment will walk you through implementing a DCGAN using the CelebA dataset in PyTorch and apply some advanced modifications.

##  Versions of used packages

We will check PyTorch version to make sure everything work properly.

I use `python 3.8.11`, `torch==1.8.2` and `torchvision==0.9.2`.

In [2]:
import sys
import torch
import torchvision
print('python', sys.version.split('\n')[0])
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)

python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
torch 1.10.1+cu113
torchvision 0.11.2+cu113


## Prepare Data

We use [CelebA Dataset](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset from cuhk university(The Chinese University of Hong Kong).

https://drive.google.com/file/d/1CYHiydUBHbnTKmZnL7M7oT_FArKMeSo-/view?usp=sharing

**Abstrct**  

CelebFaces Attributes Dataset (CelebA) is a large-scale face attributes dataset with more than 200K celebrity images, each with 40 attribute annotations.  

The images in this dataset cover large pose variations and background clutter. CelebA has large diversities, large quantities, and rich annotations, including

- 10,177 number of identities,
- 202,599 number of face images, and
- 5 landmark locations, 40 binary attributes annotations per image.
- Original Size: 218x178





## How to Get Data

Please open the file `celeba_dataset.zip`, creat shortcut to your Google Drive.

1. open [LINK of Google Drive](https://drive.google.com/file/d/1CYHiydUBHbnTKmZnL7M7oT_FArKMeSo-/view?usp=sharing)
2. Click "Add shortcut to Drive" in the top-right corner.
3. Select the location where you want to place the shortcut.
4. Click Add shortcut.

After above procedures, we have a shortcut of zip file of dataset.  
We can access this in colab after granting the permission of Google Drive.

---

請先到共用雲端硬碟將檔案 `twitter_sentiment.zip`，建立捷徑到自己的雲端硬碟中。

> 操作步驟
1. 點開雲端[連結](https://drive.google.com/file/d/1CYHiydUBHbnTKmZnL7M7oT_FArKMeSo-/view?usp=sharing)
2. 點選右上角「新增雲端硬碟捷徑」
3. 點選「我的雲端硬碟」
4. 點選「新增捷徑」

完成以上流程會在你的雲端硬碟中建立一個檔案的捷徑，接著我們在colab中取得權限即可使用。

1. Executing the below code which will provide you with an authentication link
2. Open the link
3. Choose the Google account whose Drive you want to mount
4. Allow Google Drive Stream access to your Google Account
5. Copy the code displayed, paste it in the text box as shown below, and press Enter
![](https://i1.wp.com/neptune.ai/wp-content/uploads/colab-code-copy.png?resize=512%2C102&ssl=1)

Finish!

---

執行此段後點選出現的連結，允許授權後，複製授權碼，貼在空格中後按下ENTER，即完成與雲端硬碟連結。

### Unzip Data

解壓縮 `celeba_dataset.zip` 後可以發現裡面有一個資料夾和五個 .txt 檔。

- `img_align_celeba`: contain 202599 images.
- `list_eval_partition`: image id partition for train, val and test.  
- ...

注意: 若有另外設定存放在雲端硬碟中的路徑，請記得本處路徑也須做更動。

### Load Data

In [3]:
import torch
import torch.utils.data as data
from torchvision import transforms, datasets

We can load it just use torchvision

In [5]:
def get_celeba(root, split, download=False):
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize(0.5, 0.5),
    ])
    return datasets.CelebA(root, split=split, transform=transform,
        download=download)

def get_data():
    # dataset_train = get_celeba('hw3/celeba_dataset/img_align_celebra', 'train',)
    # dataset_test = get_celeba('hw3/celeba_dataset/list_eval_partition', 'test')
    dataset_train = get_celeba('./data', 'train',)
    dataset_test = get_celeba('./data', 'test')
    print('train', len(dataset_train))
    print('test', len(dataset_test))
    return dataset_train, dataset_test

dataset_train, dataset_test = get_data()

RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [ ]:
batch_size = 32
img_size = 64
train_data = data.DataLoader(dataset_train, batch_size=batch_size,
                             drop_last=True, shuffle=True)
test_data = data.DataLoader(dataset_test, batch_size=batch_size,
                             drop_last=True)

Let's sample some images and plot them.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_imgs(img, save_file=False, file_name='out'):
    img = img.permute(1, 2, 0, 3).reshape(3, img_size, -1)
    img = (img + 1) / 2
    plt.figure(figsize=(20,10))
    plt.imshow(img.permute(1, 2, 0))
    plt.show()
    if save_file:
        plt.savefig(file_name)

plot_imgs(next(iter(train_data))[0][:8])

## Build Model
### DCGAN

[DCGAN](https://arxiv.org/abs/1511.06434) (Deep Convolutional Generative Adversarial Network) was proposed in 2015.

It use ConvTransposed(逆捲積) to build generator.

Discriminator is normal classification using Convolution.

![](https://i.imgur.com/NhERHis.png)

**DCGAN has been a excellent base strucutre of GAN using convolution.**


### SELU

There are some disadvantages in DCGAN.  
DCGAN can't be trained with high learn rate, it is easy to cause gradient vanishing.  
Recently, some research change the loss function or activation to increase the stability of GAN.    
**SELU** is a activation that was proved outperform many existing methods in Feed-Forward Neural Network.  
High-resolution DCGAN paper mentioned BatchNorm + ReLU could be replaced with SELU.

$$
\text{selu}(x) = \lambda \left\{
\begin{aligned}
    & x & if & x > 0 \\
    & \alpha e^x - \alpha & if & x \leq 0
\end{aligned}
\right. \\
\alpha = 1.6732, \lambda = 1.0507
$$


### Relativistic GAN

Relativistic GAN was proprosed in 2018.  
Old GAN discriminator only tend to make fake data more similar to real data.  
Ideal generator should also make real data fake(lost piece of original GAN) and make fake data real(original GAN).  
It will increase stability and coverage into lower reconstruction loss.

(A) 是如果用 divergence minimization 可以達到的效果，(B) 是目前 GAN 實際上用 loss 訓練，(C) 是 Relativistic GAN 的理想的優化

![](https://i.imgur.com/JxiCksr.png)

More detail please check out paper

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

latent_dim = 100
g_hidden = 128 # hidden dim of generator
d_hidden = 128 # hidden dim of discriminator

class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        # ---------- You Should Modify ----------
        # TODO: add activation
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, g_hidden * 8,
                kernel_size=4, stride=1, padding=0, bias=False),
            # --> some regularization and activation
            # shape is (g_hidden*8, 4, 4),

            nn.ConvTranspose2d(g_hidden * 8, g_hidden * 4,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (g_hidden*4, 8, 8),

            nn.ConvTranspose2d(g_hidden * 4, g_hidden * 2,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (g_hidden*2, 16, 16),

            nn.ConvTranspose2d(g_hidden * 2, g_hidden,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (g_hidden, 32, 32),

            nn.ConvTranspose2d(g_hidden, 3,
                kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh(),
            # shape is (3, 64, 64),
        )

    def forward(self, x):
        output = self.main(x)
        return output

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        # ---------- You Should Modify ----------
        # TODO: add activation
        self.main = nn.Sequential(
            nn.Conv2d(3, d_hidden,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (d_hidden, 32, 32),

            nn.Conv2d(d_hidden, d_hidden * 2,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (d_hidden * 2, 16, 16),

            nn.Conv2d(d_hidden * 2, d_hidden * 4,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (d_hidden * 4, 8, 8),

            nn.Conv2d(d_hidden * 4, d_hidden * 8,
                kernel_size=4, stride=2, padding=1, bias=False),
            # --> some regularization and activation
            # shape is (d_hidden * 8, 4, 4),

            nn.Conv2d(d_hidden * 8, 1,
                kernel_size=4, stride=1, padding=0, bias=False),
            # shape is (1, 1, 1)
        )

    def forward(self, x):
        output = self.main(x)
        return output.view(-1, 1).squeeze(1)

def weights_init(m):
    classname = m.__class__.__name__

    # ---------- You Should Modify ----------
    # TODO: correct weight initialization
    if 'Conv' in classname:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)

### Test Model
Let's test our model output shape with simulated data input.

In [ ]:
def test_model():
    net_d = Discriminator()
    net_g = Generator()

    x = torch.rand(16, latent_dim, 1, 1)

    out = net_g(x)
    print(out.shape)
    out2 = net_d(out)
    print(out2.shape)
test_model()

GAN is **computation consummed**.  
Adjust your parameter depends on your resources.

In [ ]:
# Hyper-Parameter setting
lr = 2e-4
beta1 = 0.5
start_epoch = 1
max_epoch = 3
dataset = train_data
# Suggestted setting
# train_data run 3 epoch
# test_data run 30 epoch

import os
os.makedirs('ckpts', exist_ok=True)
os.makedirs('out_imgs', exist_ok=True)

In [ ]:
device = torch.device('cuda')

net_g = Generator().to(device)
net_g.apply(weights_init) # apply weight init

net_d = Discriminator().to(device)
net_d.apply(weights_init) # apply weight init

criterion = nn.BCEWithLogitsLoss()

optim_d = torch.optim.Adam(net_d.parameters(), lr=lr, betas=(beta1, 0.99))
optim_g = torch.optim.Adam(net_g.parameters(), lr=lr, betas=(beta1, 0.99))

Make some fixed noise.  
To see evolutions of our generator model.  
And labels filled with 0 and 1 for training.

In [ ]:
fixed_noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
real_label = torch.ones(batch_size, device=device).float()
fake_label = torch.zeros(batch_size, device=device).float()

### Training Loop
Start training!

Training section of DCGAN can be split into two update stage and three parts.

**First**, we feed discirminator some true images and label 1.

**Second**, discriminator is trained with fake images generated by generator model and label 0.

Then, we do one discriminator optimizer step.

**Third**, we will make generator make some fake images and throw them into discriminator with true label 1 try to fool the discriminator.

Finally, generator optimizer update once. 

<font color="red">**BUT, remember Relativitistic GAN have different updated stage**</font>


In [ ]:
from tqdm import tqdm
import time

log_step = int(len(dataset) * 0.2)
t = time.time()

for epoch in range(start_epoch, max_epoch + 1):
    loader = enumerate(dataset)
    d_loss_list = []
    g_loss_list = []
    for i, (imgs, _) in loader:
        loss_temp = {
            'err_d': [],
            'err_g': []
        }
        imgs = imgs.to(device)
        
        # ========== Update Discriminator ==========
        # ---------- You Should Modify ----------
        net_d.zero_grad()
        
        # throw real image into discirminator
        ...

        # generate fake image and throw into discirminator
        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_imgs = net_g(noise)
        ...
        
        # calculate loss
        err_d = ...
        err_d.backward()
        loss_temp['err_d'].append(err_d.item())
        optim_d.step()
        
        # ========== Update Generator ==========
        # ---------- You Should Modify ----------
        net_g.zero_grad()
        
        # throw real image into discirminator
        ...
        
        # throw fake image into discirminator
        # noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        # fake_imgs = net_g(noise)
        ...
        
        # ---------- You Should Modify ----------
        err_g = ...
        err_g.backward()
        loss_temp['err_g'].append(err_g.item())
        optim_g.step()
        
        if i % log_step == 0:
            print('[{}/{} - {}/{}] Loss_D: {:.4f} Loss_G: {:.4f}'.format( # y_d: {:.4f} / {:.4f}
                epoch, max_epoch, i, len(dataset),
                err_d.item(), err_g.item(),# y_d_real, y_d_fake#, y_g_real, y_g_fake
            ))

            fix_fake_imgs = net_g(fixed_noise).cpu().detach()
            plot_imgs(fix_fake_imgs[:8], True, file_name='out_imgs/e{:02}.png'.format(epoch))
    
    print('-' * 30)
    l_d = sum(loss_temp['err_d']) / len(loss_temp['err_d'])
    l_g = sum(loss_temp['err_g']) / len(loss_temp['err_g'])
    d_loss_list.append(l_d)
    g_loss_list.append(l_g)
print('avg loss d', sum(d_loss_list) / len(d_loss_list))
print('avg loss d', sum(g_loss_list) / len(g_loss_list))
t = time.time() - t
print('avg time', t / max_epoch, 'secs')

## Your Task(Important)

This task of this assignment is GAN modification (based on papaer and code).

The basic of GAN is implementing a DCGAN architacture.

For the advanced modification, You can do the SELU and the relativistic GAN.

Tasks:
- Use SELU as the activation of model, fill out correct weight initialization
- Replace loss function with Repativistic GAN (RSGAN or RaSGAN are good), please read [offical GitHub](https://github.com/AlexiaJM/RelativisticGAN#to-add-relativism-to-your-own-gans-in-pytorch-you-can-use-pieces-of-code-from-below) for more details.
- Any finding, comparison of multiple methods. **Complete description (not just comparison of experiment number)**
- Any other advanced GAN is allowed (WGAN, gradient penalty, AAE...) (You can do this instead of SELU or Relativistic GAN parts)

### Points for scoring
- Full DCGAN, clear description >> **40 points**
- Add SELU activation, Relativistic GAN loss function fail, unclear description **60 points**
- Add SELU activation + Relativistic GAN loss function, clear description **100 points**
- WGAN + Relativistic GAN loss + gradient penalty, clear description **100+ points**

---

**這段很重要打中文**  

這個 lab 需要你們對 GAN 做出改善（根據論文、網路上的 code），主要是照 DCGAN 的框架下改動，最基本的是將 DCGAN 實現出來

主要修改的方向，我有提供 SELU 和 relativistic GAN 兩個方向，你可以實作這兩個，可以得到共 50 分的配分，以下有一些不同的選項可以選擇，你會拿到不同的分數

任務：
- 用 SELU 當作 model 的 activation，並補上正確的 權重初始化方法（weight initialization) 
- 修改 loss function，用 Relativistic GAN 的方法(RSGAN 或 RaSGAN 都可以)，詳情請務必參考[官方的 GitHub Code](https://github.com/AlexiaJM/RelativisticGAN#to-add-relativism-to-your-own-gans-in-pytorch-you-can-use-pieces-of-code-from-below) 
- [必要] 你的任何發現、數個方法的比較或**詳細的說明**
- **任何** GAN 的改進也非常歡迎 (WGAN, gradient penalty, AAE...) **(可以取代上面的任意分數)**

請在這裡說明你要修改的項目：DCGAN model + Relativistic GAN loss

### 我應該做多少努力
- 假設你補上 DCGAN 的 activation + loss function，加上清楚說明 >> **25 分**
- 修改成 SELU activation，Relativistic GAN loss function 失敗，未能清楚說明 >> **35 分**
- 修改成 SELU activation + Relativistic GAN loss，清楚說明 >> **50 分**
- 把架構改成 WGAN + Relativistic GAN loss，加上 gradient penalty，清楚說明 >> **60 分**